In [ ]:
def parse_input_file(filename):
    initial_values = {}
    gates_data = []
    reading_initial_values = True
    
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip empty lines
            if not line:
                reading_initial_values = False
                continue
                
            # Parse initial values section
            if reading_initial_values:
                if ':' in line:
                    wire, value = line.split(':')
                    initial_values[wire.strip()] = int(value.strip())
            # Parse gates section
            elif '->' in line:
                gates_data.append(line)
                
    return initial_values, gates_data

class Gate:
    def __init__(self, gate_type, input1, input2, output):
        self.gate_type = gate_type
        self.input1 = input1
        self.input2 = input2
        self.output = output
    
    def evaluate(self, wire_values):
        # Only evaluate if both inputs are available
        if self.input1 in wire_values and self.input2 in wire_values:
            in1 = wire_values[self.input1]
            in2 = wire_values[self.input2]
            
            if self.gate_type == "AND":
                wire_values[self.output] = 1 if in1 and in2 else 0
            elif self.gate_type == "OR":
                wire_values[self.output] = 1 if in1 or in2 else 0
            elif self.gate_type == "XOR":
                wire_values[self.output] = 1 if in1 != in2 else 0
            return True
        return False

def simulate_circuit(initial_values, gates_data):
    # Initialize wire values with initial state
    wire_values = initial_values.copy()
    
    # Parse gates
    gates = []
    for line in gates_data:
        parts = line.strip().split()
        gate_type = None
        if "AND" in line:
            gate_type = "AND"
        elif "OR" in line:
            gate_type = "OR"
        elif "XOR" in line:
            gate_type = "XOR"
        
        input1 = parts[0]
        input2 = parts[2]
        output = parts[4]
        gates.append(Gate(gate_type, input1, input2, output))
    
    # Keep evaluating gates until no more changes
    changes = True
    while changes:
        changes = False
        for gate in gates:
            if gate.evaluate(wire_values):
                changes = True
    
    # Get all z-wires in order
    z_wires = sorted([wire for wire in wire_values if wire.startswith('z')])
    
    # Print wire values for debugging
    print("\nFinal wire values:")
    for wire in sorted(wire_values.keys()):
        print(f"{wire}: {wire_values[wire]}")
    
    # Convert to binary string
    binary = ''.join(str(wire_values[wire]) for wire in z_wires)
    print(f"\nBinary result: {binary}")
    
    # Convert to decimal
    return int(binary, 2)

def main():
    # Get filename from user
    filename = input("Enter the input file name: ")
    
    try:
        # Parse input file
        initial_values, gates_data = parse_input_file(filename)
        
        # Print parsed input for verification
        print("\nInitial values:")
        for wire, value in sorted(initial_values.items()):
            print(f"{wire}: {value}")
        
        print("\nGates:")
        for gate in gates_data:
            print(gate)
        
        # Simulate circuit
        result = simulate_circuit(initial_values, gates_data)
        print(f"\nDecimal result: {result}")
        
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
    except Exception as e:
        print(f"Error processing file: {e}")

if __name__ == "__main__":
    main()